<h1>Conceito</h1>

Os métodos Naive-Bayes são uma categoria de algoritmos de apredizado supervisionados baseados em aplicar o teorema de Bayes com a suposição de que as ocorrências de cada caracterísitca são indepdentes entre si.

- O teorema de Bayes afirma que:

$
P(y | x_1,...,x_n) = \Large\frac{P(y)\ P(x_1,...,x_n  |y)}{P(x_1,...,x_n | y)}
$

- Com a consideração que a ocorrência de cada característica x é independente entre si:

$
P(x_i | y,x_1,...,x_i-1,x_i+1,...,x_n) = P(x_i,y)
$

- O que resulta em:

$
P(y | x_1,...,x_n) = \Large\frac{P(y) \prod_{i=1}^{n} P(x_i | y)}{P(x_1,...,x_n)}
$

- Como $P(x_1,...,x_n)$ é constante, a parte esquerda da equação é proporcional ao numerador da parte direita da equação. Portanto, a previsão é feita através do valor de y que maximize o numerador da parte direta da equação.

Existem diversos classificadores Naive-Bayes, sendo que estes se diferenciam nas suposições feitas sobre a distribuição de $P(x_i | y)$:

| Classificador | Suposições |
| --- | --- |
| Gaussian | Variáveis possuem distribuição normal |
| Multinomial | Variáveis possuem distribuição multinomial |
| Complement | Variáveis possuem distribuição multinomial. Se adapta bem à bases de dados desbalanceadas |
| Bernoulli | Variáveis possuem distribuição multinomial de Bernoulli |
| Categorical | Variáveis possuem distribuição categórica |

Os métodos Naive-Bayes são considerados bons classificadores, porém não são bons estimadores. Portanto, os valores calculados de probabilidade não devem ser levados tão à sério

<h1>Aplicação</h1>

In [1]:
#Importação das bibliotecas a serem utilizadas

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

In [2]:
# Importando a base de dados de treino/teste

dados = pd.read_csv('churn-bigml-80.csv')
dados.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [8]:
# Criando função para modificar os dados, formatando-os de forma adequada ao modelo

def modifica_dados(dados):

    # Adicionando colunas numéricas ao df novo
    dados_numericos = dados.loc[:, ~dados.columns.isin(['State', 'Area code', 'International plan', 'Voice mail plan', 'Churn'])].copy()

    # Formatando colunas de valores binários
    dic_binario = {
        'Yes': 1,
        'No': 0,
        True: 1,
        False: 0
    }

    dados_binarios = dados[['International plan', 'Voice mail plan', 'Churn']].replace(dic_binario)

    # Formatando colunas de dados com diversas classes através do método get_dummies
    dados_dummies = pd.get_dummies(dados[['Area code', 'State']])
    dados_dummies = dados_dummies.replace(dic_binario)

    # Concatenando os data frames
    dados_modificados = pd.concat([dados_numericos, dados_binarios, dados_dummies], axis = 1)

    return dados_modificados

In [9]:
# Transformado os dados de treino/teste

dados_modificados = modifica_dados(dados)

dados_modificados.head()

,Account length,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,...,State_SD,State_TN,State_TX,State_UT,State_VA,State_VT,State_WA,State_WI,State_WV,State_WY
0,128,25,265.1,110,45.07,197.4,99,16.78,244.7,91,...,0,0,0,0,0,0,0,0,0,0
1,107,26,161.6,123,27.47,195.5,103,16.62,254.4,103,...,0,0,0,0,0,0,0,0,0,0
2,137,0,243.4,114,41.38,121.2,110,10.30,162.6,104,...,0,0,0,0,0,0,0,0,0,0
3,84,0,299.4,71,50.90,61.9,88,5.26,196.9,89,...,0,0,0,0,0,0,0,0,0,0
4,75,0,166.7,113,28.34,148.3,122,12.61,186.9,121,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# Dividindo inputs e outputs
x = dados_modificados.drop('Churn', axis = 1).values
y = dados_modificados['Churn'].values.ravel()

In [11]:
# Normalizando inputs
norm = StandardScaler()

norm.fit(x)

x_norm = norm.transform(x)

In [12]:
# Definindo random seed no numpy

SEED = 20

np.random.seed(SEED)

In [14]:
# Dividindo dados em treino e teste:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25)

# Calculando a mediana dos valores de treino, para ser usada na variável 'binarize' do modelo. Essa variável determina um limite para binarizar os dados.
mediana = np.median(x_train)

# Instanciando o modelo, utilizando distância euclidiana
bnb = BernoulliNB(binarize = mediana)

# Realizando o treinamento
bnb.fit(x_train, y_train)

# Realizando a previsão para os dados de teste:
previsao = bnb.predict(x_test)

# Calculando o accuracy score
acuracia_bnb = accuracy_score(y_test, previsao)

print(f'Acurácia do modelo bnb treinado é: {(100*acuracia_bnb):.2f}%')

Acurácia do modelo bnb treinado é: 83.66%
